In [17]:
import os
from datetime import datetime
os.chdir("D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging")
#!pip insstall pykrige-> This is for installing the pykrige package
import pandas as pd
import numpy as np
df=pd.read_csv("python-split.csv")#This file is directly exported from STATA as data
def date_to_delta(x):#transform numerate date to datetime
    x=str(x)
    time=datetime.strptime(x,"%Y%m%d")
    return time
#import Kriging necessary package 
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
#from pykrige.uk import UniversalKriging
import matplotlib.pyplot as plt

In [ ]:
#split by pollution and date
'''pollution_set=set(df["pol"])
for p in pollution_set:
    if not os.path.exists('D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\'+p):
        os.makedirs('D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\'+p)
    sub_df=df[df["pol"]==p]        
    sub_df["date"]=sub_df["date"].apply(date_to_delta)
    sub_df=sub_df.sort_values("date",ascending=False)
    sub_df.to_csv(path_or_buf="D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\"+p+".csv")
    date=set(sub_df["date"])
    for d in date:
        sub_df_by_date=sub_df[sub_df["date"]==d]
        string_date=str(d)[:10]
        sub_df_by_date.to_csv("D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\"+p+"\\"+string_date+".csv")'''

In [ ]:
#Data prepration step
'''data = np.array(
    [
        [0.3, 1.2, 0.47],
        [1.9, 0.6, 0.56],
        [1.1, 3.2, 0.74],
        [3.3, 4.4, 1.47],
        [4.7, 3.8, 1.74],
    ]
)'''
#gridx = np.arange(x_min, x_max,(3/101))
#gridy = np.arange(y_min,y_max,(3/111))

In [ ]:
#necessary prediction points instead of gridx and gridy 
tw_grid=pd.read_csv("E:\\Kriging\\tw_grid_3km.csv")
gridx=tw_grid["x"]
gridy=tw_grid["y"]

In [ ]:
#interpolation data
no_df=pd.read_csv("NO.csv",index_col=0)
example_df=no_df[no_df["date"]=='2016-09-27']
example_df=example_df[["x","y","avg"]]
example_df=example_df.dropna()
data=np.array(example_df)

In [ ]:
OK = OrdinaryKriging(
    data[:, 0],
    data[:, 1],
    data[:, 2],
    variogram_model="exponential",
    verbose=False,
    enable_plotting=False,
)

In [ ]:
z, ss = OK.execute("points",gridx,gridy)#z for predicted value, ss for standard deviation

In [ ]:
result_df=pd.DataFrame()
result_df["x"]=gridx
result_df["y"]=gridy
result_df["predicted value"]=z
result_df["predicted standard deviation"]=ss

In [ ]:
#kt.write_asc_grid(gridx, gridy, z, filename="output.asc")
#plt.imshow(z)
#plt.show()
'''import numpy as np
ascii_grid = np.loadtxt("output.asc",skiprows=7)
ascii_grid.shape'''
#These are old output result method

In [ ]:
#compare with R output
r_df=pd.read_csv("20160928_R_out.csv")
r_df.describe()

In [ ]:
df=pd.read_csv("python-split.csv")
pollution_set=set(df["pol"])
tw_grid=pd.read_csv("D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\tw_grid_3km.csv")
gridx=tw_grid["x"]
gridy=tw_grid["y"]
for p in pollution_set:
    if not os.path.exists('D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\Python_Kriging\\'+p):
        os.makedirs('D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\Python_Kriging\\'+p)
    for v in ["avg","max","log_avg","log_max"]:
        if not os.path.exists('D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\Python_Kriging'+"\\"+p+"\\"+v):
            os.makedirs('D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\Python_Kriging'+"\\"+p+"\\"+v)
        sub_df_1=df[["date","x","y",v,"station"]]
        sub_df_1["date_delta"]=sub_df_1["date"].apply(date_to_delta)
        sub_df_1=sub_df_1.sort_values("date_delta",ascending=False)
        date_delta=set(sub_df_1["date_delta"])
        for m in ["linear", "power", "gaussian", "spherical", "exponential", "hole-effect"]:
            if not os.path.exists('D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\Python_Kriging'+"\\"+p+"\\"+v+"\\"+m):
                os.makedirs('D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\Python_Kriging'+"\\"+p+"\\"+v+"\\"+m)
            for d in date_delta:
                sub_df_2=sub_df_1[sub_df_1["date_delta"]==d]
                sub_df_2=sub_df_2.dropna(how="any")
                if len(sub_df_2)>0:
                    data=np.array(sub_df_2)
                    try:
                        OK = OrdinaryKriging(
                            data[:, 1],
                            data[:, 2],
                            data[:, 3],
                            variogram_model=m,
                            verbose=False,
                            enable_plotting=False,)
                        z, ss = OK.execute("points",gridx,gridy)
                        sub_df_3=pd.DataFrame()
                        sub_df_3["x"]=gridx
                        sub_df_3["y"]=gridy
                        sub_df_3["predicted_value"]=z
                        sub_df_3["predicted_std"]=ss
                        sub_df_3.to_csv('D:\\User_Data\\Desktop\\kan-2\\14_R_Kriging\\Python_Kriging'+"\\"+p+"\\"+v+"\\"+m+"\\"+p+"_"+v+"_"+
                           m+"_"+str(sub_df_2["date"].iloc[0])+"_"+"kriging.csv")
                    except Exception as e:
                        print(p+"_"+v+"_"+m+"_"+str(sub_df_2["date"].iloc[0])+"is not properly estimated")
                else:
                    print(str(d)+" is empty")
                

                    

                
                
            
            
            
            
    
    print(datetime.now())
            
            
            
            
            
            
            
        
        
        
        
    
    


e:\python\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


CO_avg_linear_19830703is not properly estimated
CO_avg_linear_19830726is not properly estimated
CO_avg_linear_19830402is not properly estimated
CO_avg_linear_19830531is not properly estimated
CO_avg_linear_19830709is not properly estimated
CO_avg_linear_19821022is not properly estimated
CO_avg_linear_19830504is not properly estimated
CO_avg_linear_19830521is not properly estimated
CO_avg_linear_19821209is not properly estimated
CO_avg_linear_19821111is not properly estimated
CO_avg_linear_19830403is not properly estimated
CO_avg_linear_19830607is not properly estimated
CO_avg_linear_19830115is not properly estimated
CO_avg_linear_19821003is not properly estimated
CO_avg_linear_19830629is not properly estimated
CO_avg_linear_19830628is not properly estimated
CO_avg_linear_19821128is not properly estimated
CO_avg_linear_19820723is not properly estimated
CO_avg_linear_19830714is not properly estimated
CO_avg_linear_19840527is not properly estimated
CO_avg_linear_19830421is not properly es

CO_avg_linear_19821114is not properly estimated
CO_avg_linear_19830124is not properly estimated
CO_avg_linear_19830604is not properly estimated
CO_avg_linear_19821126is not properly estimated
CO_avg_linear_19830721is not properly estimated
CO_avg_linear_19830717is not properly estimated
CO_avg_linear_19830102is not properly estimated
CO_avg_linear_19830605is not properly estimated
CO_avg_linear_19830213is not properly estimated
CO_avg_linear_19821010is not properly estimated
1982-12-21 00:00:00 is empty
1982-12-23 00:00:00 is empty
CO_avg_linear_19830810is not properly estimated
CO_avg_linear_19821011is not properly estimated
CO_avg_linear_19820712is not properly estimated
CO_avg_linear_19821225is not properly estimated
CO_avg_linear_19830212is not properly estimated
CO_avg_linear_19821030is not properly estimated
1982-08-01 00:00:00 is empty
CO_avg_linear_19830303is not properly estimated
CO_avg_linear_19830122is not properly estimated
CO_avg_linear_19830613is not properly estimated
1

CO_avg_linear_19830923is not properly estimated
CO_avg_linear_19820811is not properly estimated
CO_avg_linear_19830511is not properly estimated
CO_avg_linear_19821015is not properly estimated
CO_avg_linear_19820722is not properly estimated
CO_avg_linear_19821012is not properly estimated
CO_avg_linear_19821023is not properly estimated
CO_avg_linear_19830118is not properly estimated
CO_avg_linear_19821118is not properly estimated
CO_avg_linear_19830105is not properly estimated
CO_avg_linear_19821112is not properly estimated
CO_avg_linear_19830720is not properly estimated
CO_avg_linear_19820908is not properly estimated
CO_avg_linear_19821208is not properly estimated
CO_avg_linear_19821101is not properly estimated
CO_avg_linear_19830306is not properly estimated
CO_avg_linear_19830813is not properly estimated
CO_avg_linear_19830209is not properly estimated
CO_avg_linear_19821018is not properly estimated
CO_avg_linear_19830624is not properly estimated
CO_avg_linear_19820710is not properly es

e:\python\lib\site-packages\pykrige\core.py:652: RuntimeWarning: invalid value encountered in double_scalars
  / (np.amax(lags) - np.amin(lags)),


...

CO_max_gaussian_19830805is not properly estimated
CO_max_gaussian_19820819is not properly estimated
CO_max_gaussian_19830705is not properly estimated
1982-12-13 00:00:00 is empty
CO_max_gaussian_19830617is not properly estimated
CO_max_gaussian_19830228is not properly estimated
CO_max_gaussian_19830315is not properly estimated
CO_max_gaussian_19830310is not properly estimated
CO_max_gaussian_19821125is not properly estimated
CO_max_gaussian_19821205is not properly estimated
CO_max_gaussian_19821004is not properly estimated
CO_max_gaussian_19830417is not properly estimated
CO_max_gaussian_19830226is not properly estimated
1982-09-12 00:00:00 is empty
CO_max_gaussian_19830214is not properly estimated
CO_max_gaussian_19830316is not properly estimated
CO_max_gaussian_19830608is not properly estimated
CO_max_gaussian_19821005is not properly estimated
CO_max_gaussian_19830516is not properly estimated
CO_max_gaussian_19830415is not properly estimated
CO_max_gaussian_19830109is not properly es

CO_max_gaussian_19820817is not properly estimated
1982-07-29 00:00:00 is empty
CO_max_gaussian_19821127is not properly estimated
CO_max_gaussian_19830108is not properly estimated
CO_max_gaussian_19820727is not properly estimated
CO_max_gaussian_19830701is not properly estimated
1982-08-08 00:00:00 is empty
1983-05-07 00:00:00 is empty
CO_max_gaussian_19830730is not properly estimated
CO_max_gaussian_19830320is not properly estimated
CO_max_gaussian_19830225is not properly estimated
CO_max_gaussian_19830131is not properly estimated
CO_max_gaussian_19820827is not properly estimated
CO_max_gaussian_19830723is not properly estimated
CO_max_gaussian_19830506is not properly estimated
CO_max_gaussian_19830210is not properly estimated
CO_max_gaussian_19830806is not properly estimated
CO_max_gaussian_19820925is not properly estimated
CO_max_gaussian_19820709is not properly estimated
CO_max_gaussian_19821211is not properly estimated
CO_max_gaussian_19830330is not properly estimated
CO_max_gaussi

CO_max_spherical_19830128is not properly estimated
CO_max_spherical_19821119is not properly estimated
CO_max_spherical_19820928is not properly estimated
CO_max_spherical_19830710is not properly estimated
CO_max_spherical_19820906is not properly estimated
CO_max_spherical_19830728is not properly estimated
CO_max_spherical_19821024is not properly estimated
CO_max_spherical_19820820is not properly estimated
CO_max_spherical_19821229is not properly estimated
CO_max_spherical_19820921is not properly estimated
CO_max_spherical_19830510is not properly estimated
CO_max_spherical_19820815is not properly estimated
CO_max_spherical_19830729is not properly estimated
CO_max_spherical_19830326is not properly estimated
CO_max_spherical_19820728is not properly estimated
CO_max_spherical_19820704is not properly estimated
CO_max_spherical_19830924is not properly estimated
CO_max_spherical_19830114is not properly estimated
CO_max_spherical_19830427is not properly estimated
CO_max_spherical_19830715is not

CO_max_spherical_19830328is not properly estimated
CO_max_spherical_19820807is not properly estimated
CO_max_spherical_19820823is not properly estimated
1982-12-22 00:00:00 is empty
CO_max_spherical_19830126is not properly estimated
1982-09-10 00:00:00 is empty
CO_max_spherical_19820919is not properly estimated
CO_max_spherical_20050615is not properly estimated
CO_max_spherical_19821006is not properly estimated
CO_max_spherical_19830606is not properly estimated
CO_max_spherical_19820821is not properly estimated
CO_max_spherical_19821013is not properly estimated
CO_max_spherical_19830424is not properly estimated
CO_max_spherical_19830411is not properly estimated
CO_max_spherical_19830406is not properly estimated
CO_max_spherical_19830530is not properly estimated
CO_max_spherical_19820708is not properly estimated
CO_max_spherical_19840520is not properly estimated
CO_max_spherical_19830317is not properly estimated
CO_max_spherical_19830110is not properly estimated
CO_max_spherical_1983063

CO_max_exponential_19821111is not properly estimated
CO_max_exponential_19830403is not properly estimated
CO_max_exponential_19830607is not properly estimated
CO_max_exponential_19830115is not properly estimated
CO_max_exponential_19821003is not properly estimated
CO_max_exponential_19830629is not properly estimated
CO_max_exponential_19830628is not properly estimated
CO_max_exponential_19821128is not properly estimated
CO_max_exponential_19820723is not properly estimated
CO_max_exponential_19830714is not properly estimated
CO_max_exponential_19840527is not properly estimated
CO_max_exponential_19830421is not properly estimated
CO_max_exponential_19830319is not properly estimated
CO_max_exponential_19830519is not properly estimated
CO_max_exponential_19830514is not properly estimated
CO_max_exponential_19820903is not properly estimated
CO_max_exponential_19830202is not properly estimated
CO_max_exponential_19821227is not properly estimated
CO_max_exponential_19830329is not properly est

In [ ]:
str(sub_df_2["date"].iloc[0])

In [25]:
len(sub_df_2)

0